In [26]:
    import configparser
    from bson.objectid import ObjectId
    import pandas as pd
    import numpy as np
    import csv
    import pymongo
    import io
    import traceback
    import sys


pid = '5d4421722b3dc03043dead1e'
part_id = '5d4421722b3dc03043dead1e' 

config = configparser.ConfigParser()
config.read('configmsg.ini')
dsh = config['DASH']
MONGO_HOST = dsh['MONGO_HOST']
MONGO_PORT = int(dsh['MONGO_PORT'])
myclient = pymongo.MongoClient(MONGO_HOST,MONGO_PORT,maxPoolSize=10)
mydb = myclient[(dsh['database_name'])]

dash_usr_tbl=mydb[(dsh['dash_users'])]



In [2]:
dfs =[]

In [39]:
        part_emp_tbl=mydb['partner_employees'].find({'partner_id': ObjectId(pid)})
        loanapp_tbl=mydb['loan_applications'].find({'partner_id': ObjectId(pid)})
        user_emp_tbl=mydb['user_employments'].find({'partner_id': ObjectId(pid)})





        part_emp = pd.DataFrame(list(part_emp_tbl))
        loanapp = pd.DataFrame(list(loanapp_tbl))
        user_emp = pd.DataFrame(list(user_emp_tbl))

        part_emp['partner_id'] = part_emp['partner_id'].astype(str)
        user_emp['partner_id'] = user_emp['partner_id'].astype(str)
        loanapp['partner_id'] = loanapp['partner_id'].astype(str)
        loanapp['user_id'] = loanapp['user_id'].astype(str)

        user_emp = (user_emp[['employee_id','user_id','current','employer_name']])
        part_emp = (part_emp[['emp_id','currently_employed','department','name','registered_phone_number','work_location']])
        part_emp= part_emp.rename(columns = {'emp_id':'employee_id'})
        loanapp = loanapp[['user_id','status']]


        df = (pd.merge(user_emp, part_emp, on='employee_id'))

        df['user_id'] = df['user_id'].astype(str)
        df = (pd.merge(loanapp, df, on='user_id'))
        df = df[['name','employee_id','work_location','department','status','currently_employed','registered_phone_number','employer_name']]

        df.columns = ['Name','Emp_id','Curr_City','Department','Status','Currently_Employed','Phone_No','Vendor']
        df['Status'] = df['Status'].map({'CREDIT_DECISION_SUCCESS': 'Active','APPLIED': 'Applied'})

        df['Currently_Employed'] = df['Currently_Employed'].map({True: 'Working',False: 'Not Working'})
        dfs.append(df)
        

SyntaxError: 'return' outside function (<ipython-input-39-0fa11a8a6d76>, line 35)

In [27]:
def sub_partner_emp(pids):
    dfs=[]
    for pid in pids:

        part_emp_tbl=mydb['partner_employees'].find({'partner_id': ObjectId(pid)})
        loanapp_tbl=mydb['loan_applications'].find({'partner_id': ObjectId(pid)})
        user_emp_tbl=mydb['user_employments'].find({'partner_id': ObjectId(pid)})





        part_emp = pd.DataFrame(list(part_emp_tbl))
        loanapp = pd.DataFrame(list(loanapp_tbl))
        user_emp = pd.DataFrame(list(user_emp_tbl))

        part_emp['partner_id'] = part_emp['partner_id'].astype(str)
        user_emp['partner_id'] = user_emp['partner_id'].astype(str)
        loanapp['partner_id'] = loanapp['partner_id'].astype(str)
        loanapp['user_id'] = loanapp['user_id'].astype(str)

        user_emp = (user_emp[['employee_id','user_id','current','employer_name']])
        part_emp = (part_emp[['emp_id','currently_employed','department','name','registered_phone_number','work_location']])
        part_emp= part_emp.rename(columns = {'emp_id':'employee_id'})
        loanapp = loanapp[['user_id','status']]


        df = (pd.merge(user_emp, part_emp, on='employee_id'))

        df['user_id'] = df['user_id'].astype(str)
        df = (pd.merge(loanapp, df, on='user_id'))
        df = df[['name','employee_id','work_location','department','status','currently_employed','registered_phone_number','employer_name']]

        df.columns = ['Name','Emp_id','Curr_City','Department','Status','Currently_Employed','Phone_No','Vendor']
        df['Status'] = df['Status'].map({'CREDIT_DECISION_SUCCESS': 'Active','APPLIED': 'Applied'})

        df['Currently_Employed'] = df['Currently_Employed'].map({True: 'Working',False: 'Not Working'})
        dfs.append(df)
    return(dfs)



In [29]:
pids = ['5d4421722b3dc03043dead1e']

dfsss =[]
try:
    dfss = []
    dfss.append(sub_partner_emp(pids))
    partners_tbl=mydb['partners'].find({'parent_partner_id': ObjectId('5d4421722b3dc03043dead1e')})
    part = pd.DataFrame(list(partners_tbl))
    pids.append((part['_id'].astype(str)).tolist()) 
    dfss.append(sub_partner_emp(pids[1]))
except Exception:
    print(traceback.format_exc())

In [30]:
dfss

[[                          Name    Emp_id  Curr_City Department  Status  \
  0                     Sushitha      1003  Bengaluru          A  Active   
  1                       anisha      1001  Bengaluru          A  Active   
  2                       Shabaz  10101010   Banglore          A  Active   
  3                        Vikas  11110011   Banglore          A  Active   
  4                     Abhilash      1008  Bengaluru          A  Active   
  5               Angela Miranda      8711  Bengaluru          A  Active   
  6           Katherine Shepherd      4299  Bengaluru          A     NaN   
  7                 Alexis Reyes      4050  Bengaluru          A  Active   
  8             Jeffrey Ferguson      4042  Bengaluru          A  Active   
  9               Timothy Mccann      4368  Bengaluru          A  Active   
  10            Jennifer Jimenez      4470  Bengaluru          A  Active   
  11        Mrs. Mia Spencer PhD      4516  Bengaluru          A  Active   
  12        

In [21]:
dfss = (dfss[0]+dfss[1])

In [22]:
df = pd.concat(dfss, ignore_index=True)

In [25]:
df.loc[0:15]

,Name,Emp_id,Curr_City,Department,Status,Currently_Employed,Phone_No,Vendor
0,Sushitha,1003,Bengaluru,A,Active,Working,919945326381,LocalCompany
1,anisha,1001,Bengaluru,A,Active,Not Working,917995603016,LocalCompany
2,Shabaz,10101010,Banglore,A,Active,Working,919972115447,LocalCompany
3,Vikas,11110011,Banglore,A,Active,Working,919527119003,LocalCompany
4,Abhilash,1008,Bengaluru,A,Active,Working,919895774319,LocalCompany
5,Angela Miranda,8711,Bengaluru,A,Active,Working,918229238567,LocalCompany
6,Katherine Shepherd,4299,Bengaluru,A,Active,Working,916589146297,LocalCompany
7,Alexis Reyes,4050,Bengaluru,A,Active,Working,916565421626,LocalCompany
8,Jeffrey Ferguson,4042,Bengaluru,A,Active,Working,916546524548,LocalCompany
9,Timothy Mccann,4368,Bengaluru,A,Active,Working,916548793674,LocalCompany


In [43]:
pids = ['5d4421722b3dc03043dead1e']
dfss = []
xx = pd.DataFrame()
try:
#     dfss = []
#     a = sub_partner_emp(pids)
#     print(a)
#     dfss.append(sub_partner_emp(pids))
    print(dfss)
    partners_tbl=mydb['partners'].find({'parent_partner_id': ObjectId('5d4421722b3dc03043dead1e')})
    part = pd.DataFrame(list(partners_tbl))
    pids.append((part['_id'].astype(str)).tolist())
#     print(pids[1])
#     b = sub_partner_emp(pids[1])
#     print(b)
    dfss.append(sub_partner_emp(pids[1]))
#     print()
except Exception:
    print(traceback.format_exc())
finally:
    xx = pd.concat(dfss, ignore_index=True)
#     print(xx)
    

[]
5d7715a2a89a6353a33e5b52
5d7f6164119a9e72b46071ac


TypeError: cannot concatenate object of type "<type 'list'>"; only pd.Series, pd.DataFrame, and pd.Panel (deprecated) objs are valid

In [23]:
pids[1]

['5d7715a2a89a6353a33e5b52', '5d7f6164119a9e72b46071ac']